# Trial controllati randomizzati {#sec-rct}

**Prerequisiti**

- Consultare il primo capitolo di [Causal Inference: A Statistical Learning Approach](https://web.stanford.edu/~swager/causal_inf_book.pdf) di Stefan Wager.

**Concetti e Competenze Chiave**

**Preparazione del Notebook**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import arviz as az
import pingouin as pg
import statsmodels.api as sm

In [2]:
seed: int = sum(map(ord, "subj_prop"))
rng: np.random.Generator = np.random.default_rng(seed=seed)
sns.set_theme(palette="colorblind")
az.style.use("arviz-darkgrid")
%config InlineBackend.figure_format = "retina"

## Introduzione {.unnumbered}

Nel campo della psicologia, come in molte altre discipline scientifiche, la ricerca della causalità è fondamentale per comprendere i fenomeni umani e sviluppare interventi efficaci. Mentre la filosofia ha dibattuto per secoli sulla natura della causalità, la ricerca moderna ha fornito strumenti pratici per affrontare questa questione complessa. Tra questi, i <font color='orange'>trial controllati randomizzati</font> (RCT) emergono come un metodo particolarmente potente e affidabile.

Gli RCT rappresentano un ponte tra la complessità teorica della causalità e la sua applicazione pratica nella ricerca. Questo approccio, pionieristicamente sviluppato da studiosi come @neyman1923applications e @rubin1974estimating, permette ai ricercatori di stabilire relazioni causali con un alto grado di confidenza, grazie all'uso attento della randomizzazione e dell'analisi statistica.

Questo capitolo, che segue la trattazione presentata nel testo [Causal Inference: A Statistical Learning Approach](https://web.stanford.edu/~swager/causal_inf_book.pdf) di Stefan Wager, offre una panoramica sintetica delle stime statistiche e dell'inferenza nei trial controllati randomizzati. Quando disponibili, le prove derivate dagli RCT sono spesso considerate il "gold standard" delle evidenze statistiche; pertanto, i metodi per studiare gli RCT costituiscono la base degli strumenti statistici per l'inferenza causale. Inoltre, molti dei disegni di studio osservazionali ampiamente utilizzati, anche in psicologia, si ispirano agli RCT. Di conseguenza, questo capitolo funge anche da punto di partenza per le discussioni successive sulle stime e sull'inferenza negli studi osservazionali.

## Concetti Fondamentali negli RCT

### Effetti del Trattamento

Nel contesto degli RCT in psicologia, il "trattamento" potrebbe essere un intervento terapeutico, un programma educativo, o qualsiasi altra variabile manipolata dai ricercatori. L'obiettivo principale è determinare l'effetto di questo trattamento su una o più variabili di risultato.

### Effetti medi del trattamento (ATE e SATE)

Quando si vuole studiare l'effetto di un trattamento, una delle sfide principali è quella di capire come cambierebbe l'outcome se a una persona venisse somministrato un trattamento diverso da quello effettivamente ricevuto. Questo concetto è formalizzato nell'<font color='orange'>effetto causale individuale</font>, che è la differenza tra il risultato osservabile se una persona riceve il trattamento ($Y_i(1)$) e il risultato se non lo riceve ($Y_i(0)$). Questa differenza si esprime come:

$$
\theta_i = Y_i(1) - Y_i(0).
$$

Tuttavia, poiché per ogni individuo possiamo osservare solo uno di questi due risultati (non possiamo sapere cosa sarebbe successo se avessero ricevuto un trattamento diverso), non possiamo mai conoscere direttamente l'effetto individuale. 

Per superare questo problema, si ricorre alla media degli effetti del trattamento su un gruppo di persone, ovvero all'<font color='orange'>effetto medio del trattamento</font>. Questo può essere definito in due modi:

- **Effetto Medio Campionario del Trattamento (SATE - Sample Average Treatment Effect)**, che è la media degli effetti del trattamento nel campione:

$$
\hat{\theta} = \frac{1}{n} \sum_{i=1}^{n} (Y_i(1) - Y_i(0)).
$$

- **Effetto Medio del Trattamento nella Popolazione (ATE - Average Treatment Effect)**, che è la media dell'effetto del trattamento in un'intera popolazione da cui è tratto il campione:

$$
\tau = \mathbb{E}_P[Y_i(1) - Y_i(0)].
$$

Queste definizioni permettono di stimare l'effetto medio del trattamento anche senza osservare gli effetti individuali per ogni persona.

### Stima tramite differenza delle medie

Un metodo comune per stimare l'ATE in un trial randomizzato è usare la <font color='orange'>differenza tra le medie</font> degli outcome tra il gruppo trattato e il gruppo di controllo. Questo metodo funziona in modo molto intuitivo: calcoliamo la media dei risultati per il gruppo che ha ricevuto il trattamento e la media per il gruppo che non l'ha ricevuto, quindi sottraiamo queste due medie:

$$
\hat{\tau}_{DM} = \frac{1}{n_1} \sum_{i: W_i = 1} Y_i - \frac{1}{n_0} \sum_{i: W_i = 0} Y_i,
$$

dove $n_1$ è il numero di individui che hanno ricevuto il trattamento (gruppo trattato) e $n_0$ è il numero di individui che non lo hanno ricevuto (gruppo di controllo). 

Questo stimatore è *non distorto* se il trattamento è stato assegnato in modo casuale. Ciò significa che, in media, la stima dell'ATE fornita dalla differenza delle medie sarà corretta.

### Teorema del limite centrale e intervalli di confidenza

Per dare una misura dell'incertezza della stima dell'ATE in termini frequentisti, possiamo applicare il *teorema del limite centrale (TLC)*. Questo teorema afferma che, se il campione è sufficientemente grande e i partecipanti sono selezionati in modo indipendente da una popolazione, la distribuzione della stima dell'ATE tende a seguire una distribuzione normale, con media pari all'ATE vero e una certa varianza:

$$
\sqrt{n} (\hat{\tau}_{DM} - \tau) \sim N(0, V_{DM}),
$$

dove $V_{DM}$ rappresenta la varianza asintotica della differenza tra le medie. Questo ci permette di costruire un intervallo di confidenza per l'ATE:

$$
\hat{\tau}_{DM} \pm z_{\alpha/2} \sqrt{\frac{\hat{V}_{DM}}{n}}.
$$

Qui, $z_{\alpha/2}$ è il valore critico dalla distribuzione normale standard (ad esempio, per un livello di confidenza del 95%, $z_{0.025} = 1.96$). L'intervallo di confidenza fornisce una stima dell'ATE insieme a una misura della sua incertezza. Se l'intervallo è stretto, abbiamo una maggiore fiducia nella stima; se è largo, la stima è più incerta.

### Aggiustamenti per regressione

Gli <font color='orange'>aggiustamenti per regressione</font> permettono di migliorare la precisione della stima dell'ATE includendo informazioni aggiuntive, come covariate pre-trattamento, nel modello. L'idea di base è che, oltre al trattamento, altre variabili (covariate) potrebbero influenzare l'outcome. Quindi, includere queste covariate può aiutarci a spiegare meglio le differenze negli outcome e a ridurre la varianza della stima dell'ATE.

Vediamo passo per passo come funziona:

#### Modello di regressione con covariate
Si usa una regressione lineare che modella l'outcome $Y_i$ in funzione del trattamento $W_i$ e delle covariate $X_i$:

$$
Y_i = \alpha + W_i \tau + X_i \beta + W_i X_i \gamma + \epsilon_i,
$$

- $Y_i$: l'outcome dell'individuo $i$.
- $W_i$: il trattamento (1 se l'individuo è trattato, 0 se non lo è).
- $X_i$: le covariate pre-trattamento (es. età, genere).
- $\alpha$: l'intercetta, cioè il valore dell'outcome medio quando $W_i = 0$ e $X_i = 0$.
- $\tau$: il parametro che rappresenta l'effetto medio del trattamento (ATE), che vogliamo stimare.
- $\beta$: il vettore di parametri associati alle covariate, che misura quanto le covariate influenzano l'outcome.
- $\gamma$: il vettore di parametri di interazione tra trattamento e covariate (cioè quanto l'effetto del trattamento varia in funzione delle covariate).
- $\epsilon_i$: l'errore, che rappresenta le variazioni non spiegate dal modello.

#### Aggiustamento per covariate
La regressione permette di aggiustare gli outcome per le covariate $X_i$, separando l'effetto delle covariate dall'effetto del trattamento. Questo è utile perché riduce la "rumorosità" dei dati e rende più facile identificare l'effetto del trattamento $\tau$.

#### Stima dell'ATE
Una volta stimati i coefficienti del modello tramite una regressione (solitamente con il metodo dei minimi quadrati), possiamo calcolare la stima dell'ATE.

Il trucco è che possiamo usare i risultati della regressione per predire quale sarebbe stato l'outcome di ciascun individuo sotto entrambi i livelli del trattamento ($W_i = 1$ e $W_i = 0$):

- Per stimare l'outcome atteso se <font color='orange'>tutti gli individui fossero trattati</font> ($W_i = 1$):

  $$
  \hat{Y}_i(1) = \hat{\alpha}(1) + X_i \hat{\beta}(1).
  $$
  
- Per stimare l'outcome atteso se <font color='orange'>nessun individuo fosse trattato</font> ($W_i = 0$):

  $$
  \hat{Y}_i(0) = \hat{\alpha}(0) + X_i \hat{\beta}(0).
  $$

dove:

- $\hat{\alpha}(1)$ e $\hat{\alpha}(0)$ sono gli intercept stimati per i trattati e non trattati.
- $\hat{\beta}(1)$ e $\hat{\beta}(0)$ sono i coefficienti stimati per le covariate nei due gruppi.

L'effetto medio del trattamento (ATE) si ottiene quindi come la <font color='orange'>differenza media</font> tra questi due outcome previsti:

$$
\hat{\tau}_{IREG} = \frac{1}{n} \sum_{i=1}^{n} \left[ \hat{Y}_i(1) - \hat{Y}_i(0) \right].
$$

In pratica, l'algoritmo di regressione ti permette di stimare quale sarebbe stato l'outcome atteso per ciascun individuo se fossero stati trattati o meno, tenendo conto delle loro caratteristiche (covariate). La differenza media tra questi due scenari fornisce una stima più precisa dell'ATE.

#### Vantaggio dell'aggiustamento

L'uso delle covariate nel modello aiuta a ridurre la varianza della stima dell'ATE, poiché le covariate possono spiegare parte della variabilità negli outcome. Ciò significa che, rispetto al semplice confronto delle medie tra trattati e non trattati, l'aggiustamento per covariate porta a una stima più precisa, soprattutto quando le covariate hanno una forte influenza sugli outcome.

In conclusione, l'aggiustamento per regressione permette di stimare l'ATE in modo più efficiente, migliorando la precisione senza introdurre distorsioni, anche quando la relazione tra le covariate e gli outcome non è perfettamente lineare.

### Confronto tra stimatori

Il metodo della differenza delle medie è semplice da implementare, ma non ottimale in termini di efficienza statistica, poiché non sfrutta completamente le informazioni disponibili. Al contrario, l'aggiustamento per regressione migliora l'efficienza statistica riducendo la varianza della stima, senza introdurre distorsioni. In particolare, lo stimatore con regressione può essere sempre uguale o migliore (in termini di varianza) rispetto alla differenza delle medie, e questa riduzione della varianza diventa più significativa quando le covariate hanno una forte influenza sugli outcome.

In sintesi, l'uso della regressione per aggiustare i risultati è vantaggioso perché permette di utilizzare tutte le informazioni disponibili (inclusi i dati pre-trattamento), migliorando la precisione delle stime senza compromettere la validità delle inferenze.

Questi concetti sono fondamentali per comprendere come condurre inferenze causali in modo robusto, specialmente nei contesti sperimentali come i trial randomizzati.

## Simulazione

In questo esempio, supponiamo di avere:

- Una variabile di trattamento (0 o 1),
- Due covariate ($X_1$, $X_2$) che influenzano l'outcome,
- Un effetto causale del trattamento.

Useremo una regressione per stimare l'ATE aggiustando per le covariate.

In [3]:
# Imposta il seme per la riproducibilità
np.random.seed(42)

# Numero di osservazioni
n = 500

# Generiamo due covariate indipendenti
X1 = np.random.normal(50, 10, size=n)  # Covariata 1
X2 = np.random.normal(30, 5, size=n)  # Covariata 2

# Variabile di trattamento casuale (0 o 1)
treatment = np.random.binomial(1, 0.5, size=n)

# Effetti veri delle covariate sugli outcome
beta_X1 = 2.0
beta_X2 = -1.5

# Effetto del trattamento sull'outcome
treatment_effect = 5.0

# Generiamo l'outcome vero includendo il trattamento e le covariate
# Outcome simulato: Y = 5*T + 2*X1 - 1.5*X2 + errore
outcome = (
    treatment_effect * treatment
    + beta_X1 * X1
    + beta_X2 * X2
    + np.random.normal(0, 5, size=n)
)

# Creiamo un DataFrame per contenere i dati
data = pd.DataFrame({"Treatment": treatment, "X1": X1, "X2": X2, "Outcome": outcome})

# Mostra un'anteprima dei dati
print(data.head())

   Treatment         X1         X2    Outcome
0          0  54.967142  34.630888  56.161344
1          0  48.617357  39.547083  38.837491
2          1  56.476885  23.007162  76.707396
3          1  65.230299  32.814846  81.380258
4          0  47.658466  26.746787  61.198821


###  Stima dell'ATE senza aggiustamento (Differenza tra le medie)

La differenza tra le medie può essere calcolata confrontando l'outcome medio per il gruppo trattato e il gruppo non trattato.

In [4]:
# Calcolo della differenza tra le medie
mean_treated = data[data["Treatment"] == 1]["Outcome"].mean()
mean_control = data[data["Treatment"] == 0]["Outcome"].mean()

ate_naive = mean_treated - mean_control
print(f"ATE senza aggiustamento (differenza tra le medie): {ate_naive:.2f}")

ATE senza aggiustamento (differenza tra le medie): 9.59


### Aggiustamento per regressione

Ora eseguiamo una regressione lineare che include il trattamento e le covariate. Utilizziamo pingouin o statsmodels per eseguire la regressione e stimare l'ATE.

In [6]:
# Aggiunta di un'intercetta al modello
X = sm.add_constant(data[["Treatment", "X1", "X2"]])

# Eseguiamo la regressione con statsmodels
model = sm.OLS(data["Outcome"], X).fit()

# Mostriamo i risultati della regressione
print(model.summary())

# Estrarre il coefficiente del trattamento, che rappresenta l'ATE aggiustato
ate_adjusted = model.params["Treatment"]
print(f"\nATE con aggiustamento per le covariate: {ate_adjusted:.2f}")

                            OLS Regression Results                            
Dep. Variable:                Outcome   R-squared:                       0.952
Model:                            OLS   Adj. R-squared:                  0.951
Method:                 Least Squares   F-statistic:                     3261.
Date:                Sat, 21 Sep 2024   Prob (F-statistic):               0.00
Time:                        08:18:33   Log-Likelihood:                -1501.2
No. Observations:                 500   AIC:                             3010.
Df Residuals:                     496   BIC:                             3027.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0830      1.843     -0.045      0.9

### Interpretazione dei risultati

- **ATE senza aggiustamento (Differenza tra le medie):** Questa stima riflette l'effetto medio del trattamento senza tener conto delle covariate.
- **ATE con aggiustamento per covariate:** Questa stima corregge per le differenze nelle covariate $X_1$ e $X_2$ tra i gruppi trattati e non trattati, fornendo una stima più precisa dell'effetto causale del trattamento (nella popolazione, l'ATE simulato era pari a 5).

Questo esempio mostra come possiamo simulare una popolazione con covariate che influenzano l'outcome e come utilizzare una regressione per aggiustare queste covariate quando stimiamo l'effetto del trattamento (ATE).

## Informazioni sull'Ambiente di Sviluppo {.unnumbered}

In [43]:
%load_ext watermark
%watermark -n -u -v -iv -w -m

Last updated: Wed Jul 17 2024

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

pandas    : 2.2.2
scipy     : 1.14.0
seaborn   : 0.13.2
arviz     : 0.18.0
matplotlib: 3.9.1
numpy     : 1.26.4

Watermark: 2.4.3

